In [1]:
import os
import sys
import json

sys.path.insert(0, '../../../pipelines/tasks/mlb/')

from common.helpers.parsers import parse_game

In [2]:
dump_file = False
resave_file = False
replay = True

In [3]:
for file in os.listdir('../../../data/mlb/pbp/'):
    with open(f'../../../data/mlb/pbp/{file}', 'r', encoding='UTF8') as pbp_input:
        game = json.load(pbp_input)

    if replay:
        game = parse_game(game)

    for period in game['periods']:
        for event in period['events']:

            if not 'entities' in event:
                print(file)
                print('entities')
                print(event)
                print()
            else:
                if 'issues' in event['entities']:
                    print(file)
                    print('issues')
                    print(event)
                    print

    if replay and resave_file:
        with open(f'../../../data/mlb/pbp/{file}', 'w', encoding='UTF8') as pbp_output:
            json.dump(game, pbp_output, indent=2)

pbp_401354326.json
pbp_401354799.json
pbp_401355126.json
pbp_401427971.json
pbp_401354367.json
pbp_401355366.json
pbp_401354473.json
pbp_401355215.json
pbp_401355079.json
pbp_401355151.json
pbp_401354701.json
pbp_401355111.json
pbp_401354583.json
pbp_401355015.json
pbp_401354891.json
pbp_401355279.json
pbp_401354686.json
pbp_401355351.json
pbp_401355306.json
pbp_401354999.json
pbp_401354960.json
pbp_401354665.json
pbp_401354382.json
pbp_401354443.json
pbp_401354592.json
pbp_401355045.json
pbp_401355291.json
pbp_401355141.json
pbp_401354770.json
pbp_401354635.json
pbp_401354434.json
pbp_401355337.json
pbp_401355264.json
pbp_401355321.json
pbp_401354863.json
pbp_401354422.json
pbp_401354835.json
pbp_401354989.json
pbp_401355064.json
pbp_401354458.json
pbp_401355249.json
pbp_401354497.json
pbp_401354784.json
pbp_401354528.json
pbp_401354713.json
pbp_401355381.json
pbp_401355179.json
pbp_401354292.json
pbp_401354396.json
pbp_401354512.json
pbp_401355374.json
pbp_401355636.json
pbp_40135482

In [4]:
if dump_file:

    keys = set()
    text_instances = []

    for file in os.listdir('../../../data/mlb/pbp/'):

        with open(f'../../../data/mlb/pbp/{file}', 'r', encoding='UTF8') as pbp_input:
            game = json.load(pbp_input)

        for period in game['periods']:
            for event in period['events']:

                description = event['desc']
                if not description in keys:
                    text_instances.append((description, event['entities']))
                    keys.add(description)

    with open('../../../pipelines/tasks/mlb/common/tests/docs/desc_to_entities.json', 'w') as test_instances:
        json.dump(text_instances, test_instances, indent=2)

In [5]:
### take entities and remove from description ... should be finite unique???
